# Data Download

In [ ]:
from pathlib import Path

import cdsapi
from loguru import logger
from tqdm import tqdm

from compstat_research.config import INTERIM_DATA_DIR

currentPath = Path()
savePath = INTERIM_DATA_DIR / "cds" / "era5" / "2m-temperature"

if not savePath.exists():
    savePath.mkdir(parents=True)

years = range(1940, 2025)

logger.add("1.0-iragca-cds-data.log", rotation="500 MB", retention="7 days", compression="zip")
logger.info(f"Starting download of ERA5 data from {years[0]} to {years[-1]}.")
for year in tqdm(years, desc="Downloading data", unit="year"):
    try:
        dataset = "derived-era5-single-levels-daily-statistics"
        request = {
            "product_type": "reanalysis",
            "variable": ["2m_temperature"],
            "year": f"{year}",
            "month": [
                "01", "02", "03",
                "04", "05", "06",
                "07", "08", "09",
                "10", "11", "12"
            ],
            "day": [
                "01", "02", "03",
                "04", "05", "06",
                "07", "08", "09",
                "10", "11", "12",
                "13", "14", "15",
                "16", "17", "18",
                "19", "20", "21",
                "22", "23", "24",
                "25", "26", "27",
                "28", "29", "30",
                "31"
            ],
            "daily_statistic": "daily_mean",
            "time_zone": "utc+08:00",
            "frequency": "1_hourly",
            "area": [20, 115, 5, 130]
        }

        client = cdsapi.Client()
        client.retrieve(dataset, request).download()

        for file in currentPath.iterdir():
            if file.is_file() and file.suffix == ".nc":
                file.rename(savePath / f"{year}-{file.name}")
                logger.info(f"Moved file {file.name} to {savePath}.")

    except Exception as e:
        logger.error(f"Error downloading data for year {year}. Error: {e}")
        continue
    else:
        logger.info(f"Downloaded data for year {year}.")

In [ ]:
# Custom download
try:
    customYear = 1996
    dataset = "derived-era5-single-levels-daily-statistics"
    request = {
        "product_type": "reanalysis",
        "variable": ["2m_temperature"],
        "year": f"{customYear}",
        "month": [
            "01", "02", "03",
            "04", "05", "06",
            "07", "08", "09",
            "10", "11", "12"
        ],
        "day": [
            "01", "02", "03",
            "04", "05", "06",
            "07", "08", "09",
            "10", "11", "12",
            "13", "14", "15",
            "16", "17", "18",
            "19", "20", "21",
            "22", "23", "24",
            "25", "26", "27",
            "28", "29", "30",
            "31"
        ],
        "daily_statistic": "daily_mean",
        "time_zone": "utc+08:00",
        "frequency": "1_hourly",
        "area": [20, 115, 5, 130]
    }

    client = cdsapi.Client()
    client.retrieve(dataset, request).download()

    for file in currentPath.iterdir():
        if file.is_file() and file.suffix == ".nc":
            file.rename(savePath / file.name)
            logger.info(f"Moved file {file.name} to {savePath}.")
except Exception as e:
    logger.error(f"Error downloading data for year {customYear}. Error: {e}")
else:
    logger.info(f"Downloaded data for year {customYear}.")

##  Data Aggregation

In [1]:
from pathlib import Path

import polars as pl
import xarray as xr
from loguru import logger
from tqdm import tqdm

from compstat_research.config import INTERIM_DATA_DIR, PROCESSED_DATA_DIR

2025-02-09 19:30:03.112 | INFO     | compstat_research.config:<module>:11 - PROJ_ROOT path is: D:\Documents\3 Production Save Files\1 Cloned repositories\compstat-research


In [ ]:
currentPath = Path()
savePath = INTERIM_DATA_DIR / "cds" / "era5" / "2m-temperature"

if not savePath.exists():
    savePath.mkdir(parents=True)

logger.add("1.0-iragca-cds-data.log", rotation="500 MB", retention="7 days", compression="zip")
for file in currentPath.iterdir():
    if file.is_file() and file.suffix == ".nc":
        file.rename(savePath / file.name)
        logger.info(f"Moved file {file.name} to {savePath}.")

2025-02-09 19:33:23.611 | INFO     | __main__:<module>:11 - Moved file 11376002cd7f0ebab274a02b3e7b87af.nc to D:\Documents\3 Production Save Files\1 Cloned repositories\compstat-research\data\interim\cds\era5\2m-temperature.
2025-02-09 19:33:23.616 | INFO     | __main__:<module>:11 - Moved file 13f1e53413d59f05a4f701f874e5d0f9.nc to D:\Documents\3 Production Save Files\1 Cloned repositories\compstat-research\data\interim\cds\era5\2m-temperature.
2025-02-09 19:33:23.616 | INFO     | __main__:<module>:11 - Moved file 150f29fa89d81a023d34470043f577a5.nc to D:\Documents\3 Production Save Files\1 Cloned repositories\compstat-research\data\interim\cds\era5\2m-temperature.
2025-02-09 19:33:23.617 | INFO     | __main__:<module>:11 - Moved file 17e15151c2663f11efac6b8fcb6090ff.nc to D:\Documents\3 Production Save Files\1 Cloned repositories\compstat-research\data\interim\cds\era5\2m-temperature.
2025-02-09 19:33:23.618 | INFO     | __main__:<module>:11 - Moved file 18364ada71922ea5af598ad68de48

In [15]:
# Concatenate all files into a single DataFrame
currentPath = Path()
df = pl.DataFrame()
logger.add("1.0-iragca-cds-data.log", rotation="500 MB", retention="7 days", compression="zip")

for file in currentPath.iterdir():
    try:
        if file.suffix == ".nc":
            logger.info(f"Reading file {file}")
            current_df = xr.open_dataset(file)
            df = pl.concat([df, pl.DataFrame(current_df.to_dataframe().reset_index())], how="vertical")
            logger.info(f"File {file} read successfully.")
    except Exception as e:
        logger.error(f"Error reading file {file}. Error: {e}")
        continue
        

2025-02-08 21:12:37.187 | INFO     | __main__:<module>:8 - Reading file 11376002cd7f0ebab274a02b3e7b87af.nc
2025-02-08 21:12:37.719 | INFO     | __main__:<module>:11 - File 11376002cd7f0ebab274a02b3e7b87af.nc read successfully.
2025-02-08 21:12:37.721 | INFO     | __main__:<module>:8 - Reading file 13f1e53413d59f05a4f701f874e5d0f9.nc
2025-02-08 21:12:37.869 | INFO     | __main__:<module>:11 - File 13f1e53413d59f05a4f701f874e5d0f9.nc read successfully.
2025-02-08 21:12:37.870 | INFO     | __main__:<module>:8 - Reading file 150f29fa89d81a023d34470043f577a5.nc
2025-02-08 21:12:38.020 | INFO     | __main__:<module>:11 - File 150f29fa89d81a023d34470043f577a5.nc read successfully.
2025-02-08 21:12:38.022 | INFO     | __main__:<module>:8 - Reading file 17e15151c2663f11efac6b8fcb6090ff.nc
2025-02-08 21:12:38.243 | INFO     | __main__:<module>:11 - File 17e15151c2663f11efac6b8fcb6090ff.nc read successfully.
2025-02-08 21:12:38.244 | INFO     | __main__:<module>:8 - Reading file 18364ada71922ea5

In [21]:
df = df.sort("valid_time")
df

valid_time,latitude,longitude,t2m,number
datetime[ns],f64,f64,f32,i64
1940-01-02 00:00:00,20.0,115.0,292.075531,0
1940-01-02 00:00:00,20.0,115.25,292.206451,0
1940-01-02 00:00:00,20.0,115.5,292.347656,0
1940-01-02 00:00:00,20.0,115.75,292.450195,0
1940-01-02 00:00:00,20.0,116.0,292.558685,0
…,…,…,…,…
2024-12-31 00:00:00,5.0,129.0,300.950104,0
2024-12-31 00:00:00,5.0,129.25,300.983002,0
2024-12-31 00:00:00,5.0,129.5,301.038422,0


In [24]:
df.drop_in_place('number')
df

valid_time,latitude,longitude,t2m
datetime[ns],f64,f64,f32
1970-01-01 00:00:00,20.0,115.0,294.727814
1970-01-01 00:00:00,20.0,115.25,294.821075
1970-01-01 00:00:00,20.0,115.5,294.898224
1970-01-01 00:00:00,20.0,115.75,294.938843
1970-01-01 00:00:00,20.0,116.0,294.978729
…,…,…,…
1964-12-31 00:00:00,5.0,129.0,299.857208
1964-12-31 00:00:00,5.0,129.25,299.905792
1964-12-31 00:00:00,5.0,129.5,299.953064


In [18]:
df.estimated_size() / (1024 * 1024) # megabytes

3966.138816833496

In [28]:
# Save the files
save_path = PROCESSED_DATA_DIR / "cds"

if not save_path.exists():
    save_path.mkdir(parents=True)

df.write_parquet(save_path / "era5-2m-temperature-1940-2024.parquet")

: 

In [1]:
%reset -f
import gc

import xarray as xr
from loguru import logger

from compstat_research.config import INTERIM_DATA_DIR, PROCESSED_DATA_DIR

data_path = INTERIM_DATA_DIR / "cds" / "era5" / "2m-temperature"
save_path = PROCESSED_DATA_DIR / "cds" / "era5" / "2m-temperature" / "global"

if not save_path.exists():
    save_path.mkdir(parents=True)

logger.add("1.0-iragca-cds-data.log", rotation="500 MB", retention="7 days", compression="zip")
logger.info(f"Starting to process files in {data_path}. Saving to {save_path}.")

for year in range(1959, 2025):
    for file in data_path.iterdir():
        try:
            if (file.suffix == ".nc") and (file.name.startswith(f"global-{year}")):
                logger.info(f"Processing file {file.name}.")
                current_df = (
                    xr.open_dataset(file)
                    .to_dataframe()
                    .reset_index()
                )
                current_df.to_parquet(save_path / f"{file.stem}.parquet", engine="pyarrow")
                logger.success(f"Saved file {file.name} to {save_path}.")
                del current_df
        except Exception as e:
            logger.error(f"Error processing file {file.name}. Error: {e}")
        finally:
            gc.collect()

2025-02-16 17:32:30.533 | INFO     | compstat_research.config:<module>:11 - PROJ_ROOT path is: D:\Documents\3 Production Save Files\1 Cloned repositories\compstat-research


2025-02-16 17:32:30.622 | INFO     | __main__:<module>:16 - Starting to process files in D:\Documents\3 Production Save Files\1 Cloned repositories\compstat-research\data\interim\cds\era5\2m-temperature. Saving to D:\Documents\3 Production Save Files\1 Cloned repositories\compstat-research\data\processed\cds\era5\2m-temperature\global.
2025-02-16 17:32:33.051 | INFO     | __main__:<module>:22 - Processing file global-1959-b960eadc6b78cad5ce55f3e096ea783c.nc.
2025-02-16 17:33:56.700 | SUCCESS  | __main__:<module>:29 - Saved file global-1959-b960eadc6b78cad5ce55f3e096ea783c.nc to D:\Documents\3 Production Save Files\1 Cloned repositories\compstat-research\data\processed\cds\era5\2m-temperature\global.
2025-02-16 17:34:02.656 | INFO     | __main__:<module>:22 - Processing file global-1960-668e97e261d3ee5f5c36d6db56c5af70.nc.
2025-02-16 17:35:45.710 | SUCCESS  | __main__:<module>:29 - Saved file global-1960-668e97e261d3ee5f5c36d6db56c5af70.nc to D:\Documents\3 Production Save Files\1 Clone

In [5]:
%reset -f
import gc

import polars as pl
import xarray as xr
from IPython.display import clear_output
from loguru import logger

from compstat_research.config import INTERIM_DATA_DIR, PROCESSED_DATA_DIR

data_path = INTERIM_DATA_DIR / "cds" / "era5" / "2m-temperature"
save_path = PROCESSED_DATA_DIR / "cds" / "era5" / "2m-temperature" / "global"
file = data_path / "global-1941-8637bf174e20f8fe4de25c68d1abca34.nc"

if not save_path.exists():
    save_path.mkdir(parents=True)

gc.collect()
clear_output(wait=True)

logger.info("Opening xarray dataset.")
data = xr.open_dataset(file)
logger.success("Xarray dataset opened successfully.")

logger.info("Converting xarray to polars DataFrame.")
pl_data = data.to_dataframe().reset_index()
pl_data.to_parquet(save_path / f"{file.stem}.parquet")
logger.success("Saved file to parquet successfully.")
del pl_data

2025-02-16 09:16:48.451 | INFO     | __main__:<module>:21 - Opening xarray dataset.
2025-02-16 09:16:48.510 | SUCCESS  | __main__:<module>:23 - Xarray dataset opened successfully.
2025-02-16 09:16:48.510 | INFO     | __main__:<module>:25 - Converting xarray to polars DataFrame.
2025-02-16 09:17:57.501 | SUCCESS  | __main__:<module>:28 - Saved file to parquet successfully.


In [4]:
# del current_df
from IPython.display import clear_output
import gc
gc.collect()
clear_output(wait=True)

## Database Insertion

In [1]:
import duckdb
import polars as pl

from loguru import logger

from compstat_research.config import PROCESSED_DATA_DIR

logger.info("Connecting to DuckDB.")
save_path = PROCESSED_DATA_DIR / "cds" / "era5" / "2m-temperature" / "global"
conn = duckdb.connect(save_path / "era5-2m-temperature-global.duckdb")
logger.success("Connected to DuckDB.")

logger.info("Creating temperature table.")
TEMPERATURE_TABLE = """
CREATE OR REPLACE TABLE temperature (
    valid_time TIMESTAMP,
    latitude FLOAT,
    longitude FLOAT,
    t2m FLOAT
  );
"""
conn.sql(TEMPERATURE_TABLE)
logger.success("Temperature table created successfully.")

logger.info("Reading data from parquet file.")
data = pl.read_parquet(save_path / "global-1940-13a2f5ed69a607563a645692f808895.parquet", columns=["valid_time", "latitude", "longitude", "t2m"])
logger.success("Data read successfully.")

logger.info("Inserting data into temperature table.")
conn.sql("INSERT INTO temperature SELECT * FROM data")
logger.success("Data inserted successfully.")
del data

logger.info("Querying data from temperature table.")
conn.sql("SELECT * FROM temperature LIMIT 10")
logger.success("Data queried successfully.")

conn.commit()
conn.close()
logger.success("Connection closed successfully.")

2025-02-16 19:45:53.554 | INFO     | compstat_research.config:<module>:11 - PROJ_ROOT path is: D:\Documents\3 Production Save Files\1 Cloned repositories\compstat-research


2025-02-16 19:45:53.640 | INFO     | __main__:<module>:8 - Connecting to DuckDB.
2025-02-16 19:45:54.322 | SUCCESS  | __main__:<module>:11 - Connected to DuckDB.
2025-02-16 19:45:54.322 | INFO     | __main__:<module>:13 - Creating temperature table.
2025-02-16 19:45:54.414 | SUCCESS  | __main__:<module>:23 - Temperature table created successfully.
2025-02-16 19:45:54.414 | INFO     | __main__:<module>:25 - Reading data from parquet file.
2025-02-16 19:46:03.048 | SUCCESS  | __main__:<module>:27 - Data read successfully.
2025-02-16 19:46:03.048 | INFO     | __main__:<module>:29 - Inserting data into temperature table.


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

2025-02-16 19:46:37.711 | SUCCESS  | __main__:<module>:31 - Data inserted successfully.
2025-02-16 19:46:37.713 | INFO     | __main__:<module>:34 - Querying data from temperature table.
2025-02-16 19:46:37.793 | SUCCESS  | __main__:<module>:36 - Data queried successfully.
2025-02-16 19:46:38.190 | SUCCESS  | __main__:<module>:40 - Connection closed successfully.


In [10]:
conn = duckdb.connect(save_path / "era5-2m-temperature-global.duckdb")

print(conn.sql("SELECT * FROM temperature LIMIT 50"))
conn.close()

┌─────────────────────┬──────────┬───────────┬──────────┐
│     valid_time      │ latitude │ longitude │   t2m    │
│      timestamp      │  float   │   float   │  float   │
├─────────────────────┼──────────┼───────────┼──────────┤
│ 1940-01-02 00:00:00 │     90.0 │       0.0 │ 248.6584 │
│ 1940-01-02 00:00:00 │     90.0 │      0.25 │ 248.6584 │
│ 1940-01-02 00:00:00 │     90.0 │       0.5 │ 248.6584 │
│ 1940-01-02 00:00:00 │     90.0 │      0.75 │ 248.6584 │
│ 1940-01-02 00:00:00 │     90.0 │       1.0 │ 248.6584 │
│ 1940-01-02 00:00:00 │     90.0 │      1.25 │ 248.6584 │
│ 1940-01-02 00:00:00 │     90.0 │       1.5 │ 248.6584 │
│ 1940-01-02 00:00:00 │     90.0 │      1.75 │ 248.6584 │
│ 1940-01-02 00:00:00 │     90.0 │       2.0 │ 248.6584 │
│ 1940-01-02 00:00:00 │     90.0 │      2.25 │ 248.6584 │
│          ·          │       ·  │        ·  │     ·    │
│          ·          │       ·  │        ·  │     ·    │
│          ·          │       ·  │        ·  │     ·    │
│ 1940-01-02 0

In [1]:
# Run this script to create the temperature table (OR RESET THE TABLE!)
# in a duckdb database from all parquet files
# in the save path.

import duckdb
import polars as pl

from loguru import logger

from compstat_research.config import PROCESSED_DATA_DIR

logger.info("Connecting to DuckDB.")
save_path = PROCESSED_DATA_DIR / "cds" / "era5" / "2m-temperature" / "global"
conn = duckdb.connect(save_path / "era5-2m-temperature-global.duckdb")
logger.success("Connected to DuckDB.")

logger.info("Creating temperature table.")
TEMPERATURE_TABLE = """
CREATE OR REPLACE TABLE temperature (
    valid_time TIMESTAMP,
    latitude FLOAT,
    longitude FLOAT,
    t2m FLOAT
  );
"""
conn.sql(TEMPERATURE_TABLE)
logger.success("Temperature table created successfully.")
conn.close()

2025-02-16 19:58:44.591 | INFO     | compstat_research.config:<module>:11 - PROJ_ROOT path is: D:\Documents\3 Production Save Files\1 Cloned repositories\compstat-research


2025-02-16 19:58:44.602 | INFO     | __main__:<module>:8 - Connecting to DuckDB.
2025-02-16 19:58:44.631 | SUCCESS  | __main__:<module>:11 - Connected to DuckDB.
2025-02-16 19:58:44.632 | INFO     | __main__:<module>:13 - Creating temperature table.
2025-02-16 19:58:44.889 | SUCCESS  | __main__:<module>:23 - Temperature table created successfully.


In [2]:
# This script is used to insert data into the 
# temperature table in a duckdb database from 
# all parquet files in the save path.

import duckdb
import polars as pl

from loguru import logger

from compstat_research.config import PROCESSED_DATA_DIR

save_path = PROCESSED_DATA_DIR / "cds" / "era5" / "2m-temperature" / "global"

logger.add("1.0-iragca-cds-data.log", rotation="500 MB", retention="7 days", compression="zip")

# Customize the year range to be iterated over
for year in range(1940, 2025):
    # Iterate over all parquet files in the save path
    for file in save_path.iterdir():
        if (file.suffix == ".parquet") and (file.name.startswith(f"global-{year}")):
            # Connect to DuckDB
            logger.info("Connecting to DuckDB.")
            save_path = PROCESSED_DATA_DIR / "cds" / "era5" / "2m-temperature" / "global"
            conn = duckdb.connect(save_path / "era5-2m-temperature-global.duckdb")
            logger.success("Connected to DuckDB.")

            # Read data from parquet file
            logger.info(f"Reading data from parquet file {file.name}.")
            data = pl.read_parquet(file, columns=["valid_time", "latitude", "longitude", "t2m"])
            logger.success("Data read successfully.")

            # Insert data into temperature table
            logger.info("Inserting data into temperature table.")
            conn.sql("INSERT INTO temperature SELECT * FROM data")
            logger.success("Data inserted successfully.")
            
            # Delete data from memory and make space for the next .parquet file
            del data

            # Query data from temperature table
            logger.info("Querying data from temperature table.")
            conn.sql("SELECT * FROM temperature LIMIT 10")
            logger.success("Data queried successfully.")

            # Commit and close connection
            conn.commit()
            conn.close()
            logger.success("Connection closed successfully.")

2025-02-16 20:00:59.678 | INFO     | __main__:<module>:22 - Connecting to DuckDB.
2025-02-16 20:00:59.707 | SUCCESS  | __main__:<module>:25 - Connected to DuckDB.
2025-02-16 20:00:59.707 | INFO     | __main__:<module>:28 - Reading data from parquet file global-1940-13a2f5ed69a607563a645692f808895.parquet.
2025-02-16 20:01:00.849 | SUCCESS  | __main__:<module>:30 - Data read successfully.
2025-02-16 20:01:00.850 | INFO     | __main__:<module>:33 - Inserting data into temperature table.


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

2025-02-16 20:01:18.807 | SUCCESS  | __main__:<module>:35 - Data inserted successfully.
2025-02-16 20:01:18.809 | INFO     | __main__:<module>:41 - Querying data from temperature table.
2025-02-16 20:01:18.809 | SUCCESS  | __main__:<module>:43 - Data queried successfully.
2025-02-16 20:01:19.144 | SUCCESS  | __main__:<module>:48 - Connection closed successfully.
2025-02-16 20:01:19.145 | INFO     | __main__:<module>:22 - Connecting to DuckDB.
2025-02-16 20:01:19.165 | SUCCESS  | __main__:<module>:25 - Connected to DuckDB.
2025-02-16 20:01:19.165 | INFO     | __main__:<module>:28 - Reading data from parquet file global-1941-8637bf174e20f8fe4de25c68d1abca34.parquet.
2025-02-16 20:02:32.673 | SUCCESS  | __main__:<module>:30 - Data read successfully.
2025-02-16 20:02:32.674 | INFO     | __main__:<module>:33 - Inserting data into temperature table.


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

2025-02-16 20:02:52.217 | SUCCESS  | __main__:<module>:35 - Data inserted successfully.
2025-02-16 20:02:52.218 | INFO     | __main__:<module>:41 - Querying data from temperature table.
2025-02-16 20:02:52.219 | SUCCESS  | __main__:<module>:43 - Data queried successfully.
2025-02-16 20:02:52.824 | SUCCESS  | __main__:<module>:48 - Connection closed successfully.
2025-02-16 20:02:52.825 | INFO     | __main__:<module>:22 - Connecting to DuckDB.
2025-02-16 20:02:52.846 | SUCCESS  | __main__:<module>:25 - Connected to DuckDB.
2025-02-16 20:02:52.846 | INFO     | __main__:<module>:28 - Reading data from parquet file global-1942-eb70c65941c4e31671ecb46947961807.parquet.


KeyboardInterrupt: 